
Imports e definição do dataset.

In [28]:
import tensorflow as tf
import time
from tensorflow import keras
import matplotlib.pyplot as plt
import pprint

In [23]:
datasets = [
    keras.datasets.mnist,
    keras.datasets.fashion_mnist,
    keras.datasets.cifar10,
    keras.datasets.cifar100
]
#Carrega duas tuplas, representando os dados de treinamento e de teste.
#Cada tupla tem as imagens e os respectivos rótulos
all_data = [dataset.load_data() for dataset in datasets]
all_train = [data[0] for data in all_data]
all_test = [data[1] for data in all_data]

Tarefa
Escreva código para executar redes neurais nos seguintes datasets:

MNIST (pode aproveitar o codigo existente)
Fashion MNIST
CIFAR-10
CIFAR-100
Cada execução deve ser por 10 épocas.

Você deve preencher as funções a seguir para retornarem a rede neural com a melhor configuração que você conseguiu para cada dataset. O notebook deve ser entregue com a rede neural que obteve a melhor performance em cada conjunto de dados.

IMPORTANTE: as funções não devem TREINAR nem AVALIAR as redes neurais, apenas instanciá-las e retorná-las.

Ao final, preencha o dict results com o desempenho encontrado em cada execução.

In [24]:
def get_mnist_network():
  num_classes = 10
  model = keras.Sequential([
      tf.keras.layers.Input(shape=(28, 28, 1)),

      tf.keras.layers.Conv2D(16, (3, 3), activation='gelu', padding='same'),
      tf.keras.layers.Conv2D(16, (3, 3), activation='gelu', padding='same'),

      tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Dropout(0.25),

      tf.keras.layers.Flatten(),

      tf.keras.layers.Dense(50, activation='gelu'),
      tf.keras.layers.Dense(50, activation='gelu'),
      tf.keras.layers.Dropout(0.5),

      tf.keras.layers.Dense(num_classes, activation='softmax')
  ])

  model.compile(optimizer='adamw',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

def get_fashion_mnist_network():
  num_classes = 10
  model = keras.Sequential([
      tf.keras.layers.Input(shape=(28, 28, 1)),

      tf.keras.layers.Conv2D(16, (3, 3), activation='gelu', padding='same'),
      tf.keras.layers.Conv2D(16, (3, 3), activation='gelu', padding='same'),

      tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Dropout(0.25),

      tf.keras.layers.Flatten(),

      tf.keras.layers.Dense(50, activation='gelu'),
      tf.keras.layers.Dense(50, activation='gelu'),
      tf.keras.layers.Dropout(0.5),

      tf.keras.layers.Dense(num_classes, activation='softmax')
  ])

  model.compile(optimizer='adamw',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

def get_cifar10_network():
  num_classes = 10
  model = keras.Sequential([
      tf.keras.layers.Input(shape=(32, 32, 3)),
      tf.keras.layers.BatchNormalization(),

      tf.keras.layers.Conv2D(32, (3, 3), activation='gelu', padding='same'),
      tf.keras.layers.Conv2D(32, (3, 3), activation='gelu', padding='same'),

      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Dropout(0.25),

      tf.keras.layers.Conv2D(64, (3, 3), activation='gelu', padding='same'),
      tf.keras.layers.Conv2D(64, (3, 3), activation='gelu', padding='same'),

      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Dropout(0.25),

      tf.keras.layers.Conv2D(128, (3, 3), activation='gelu', padding='same'),
      tf.keras.layers.Conv2D(128, (3, 3), activation='gelu', padding='same'),

      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Dropout(0.25),

      tf.keras.layers.Flatten(),

      tf.keras.layers.Dense(256, activation='gelu'),

      tf.keras.layers.Dropout(0.5),

      tf.keras.layers.Dense(num_classes, activation='softmax')
  ])

  model.compile(optimizer='adamw',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

def get_cifar100_network():
  num_classes = 100
  model = keras.Sequential([
      tf.keras.layers.Input(shape=(32, 32, 3)),
      tf.keras.layers.BatchNormalization(),

      tf.keras.layers.SeparableConv2D(128, (3, 3), activation='gelu', padding='same'),

      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Dropout(0.25),

      tf.keras.layers.Conv2D(128, (3, 3), activation='gelu', padding='same'),
      tf.keras.layers.Conv2D(128, (3, 3), activation='gelu', padding='same'),

      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Dropout(0.25),

      tf.keras.layers.Conv2D(256, (3, 3), activation='gelu', padding='same'),
      tf.keras.layers.Conv2D(256, (3, 3), activation='gelu', padding='same'),

      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Dropout(0.25),

      tf.keras.layers.Flatten(),

      tf.keras.layers.Dense(512, activation='gelu'),
      tf.keras.layers.Dense(512, activation='gelu'),

      tf.keras.layers.Dropout(0.5),

      tf.keras.layers.Dense(num_classes, activation='softmax')
  ])

  model.compile(optimizer='adamw',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model


In [25]:
model_generators = [
    get_mnist_network,
    get_fashion_mnist_network,
    get_cifar10_network,
    get_cifar100_network
]

def time_func(func):
  start_time = time.perf_counter()
  func()
  end_time = time.perf_counter()
  return end_time - start_time

def train_model(index, verbose=False):
    model = model_generators[index]()

    if verbose:
      model.summary()
    verbose = 1 if verbose else 0

    train_time = time_func(lambda: model.fit(*all_train[index], verbose=verbose, epochs=10))


    # Avalie o modelo no conjunto de teste
    test_loss, test_accuracy = model.evaluate(*all_test[index], verbose=verbose)
    print(f'Acurácia no conjunto de teste: {test_accuracy * 100:.2f}%')
    print(f'Tempo de treino: {train_time:.2f}s')
    return {"model": model, "acc": test_accuracy, "time": train_time}

In [30]:
#all_models = enumerate(["mnist", "fashion_mnist", "cifar10", "cifar100"])
all_models = [(0, "mnist")]
results = {
    nome: {"time": result["time"], "acc": result["acc"] } for i, nome in all_models if (result := train_model(i, verbose=True))
}
pprint.pprint(results)

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_28 (Conv2D)              │ (None, 28, 28, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 28, 28, 16)     │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 14, 14, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 14, 14, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 50)             │       156,850 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 50)             │         2,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 10)             │           510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 162,390 (634.34 KB)

 Trainable params: 162,390 (634.34 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.6435 - loss: 1.7194
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9628 - loss: 0.1462
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9720 - loss: 0.1142
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9781 - loss: 0.0843
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9812 - loss: 0.0750
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9824 - loss: 0.0656
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9843 - loss: 0.0576
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9858 - loss: 0.0555
Epoch 9/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9865 - loss: 0.0514
Epoch 10/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9851 - loss: 0.0545
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9827 - loss: 0.0784
Acurácia no conjunto de teste: 98.53%
Tempo

Preencha o dict abaixo substituindo os None com a acuracia final (acc) e o tempo de treinamento (time) encontrado no seu experimento pra cada dataset

In [27]:
results = {
    'mnist':          {'time': 63.85032578000005 , 'acc': 0.9866999983787537},
    'fashion_mnist':  {'time': 69.17781601399997 , 'acc': 0.9089000225067139},
    'cifar10':        {'time': 102.44512897700008, 'acc': 0.8087000250816345},
    'cifar100':       {'time': 175.61647450700002, 'acc': 0.4278999865055084}
}